In [2]:
import torch
from sklearn.model_selection import train_test_split

In [3]:
with open('input.txt', 'r', encoding = "utf-8") as file:
    text = file.read()

In [4]:
print(len(text))
vocab = len(sorted(set(text)))
print(vocab)

1115393
65


In [5]:
#find all unique characters in the text
chars = sorted(list(set(text)))
print('total chars:',chars,  len(chars))

total chars: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [6]:
#create tokens for each char
tokens = {char: idx for idx, char in enumerate(set(text))}
snekot = {idx: char for idx, char in enumerate(set(text))}
encode = lambda s: [tokens[c] for c in s]
decode = lambda a: ''.join([snekot[i] for i in a])
print(encode("Hello there"))
print(decode(encode("Hello there")))


[28, 22, 61, 61, 27, 10, 19, 48, 22, 32, 22]
Hello there


In [7]:
import sentencepiece as spm
spm.SentencePieceTrainer.train(input='input.txt', model_prefix='model', vocab_size=10000)


In [8]:
sp = spm.SentencePieceProcessor(model_file='model.model')
sp.Encode("My name is John.")
sp.DecodeIds([89, 212, 25, 1370, 5])

'My name is John.'

In [9]:
data = torch.tensor(encode(text), dtype=torch.int64)

In [10]:
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

In [11]:
block_size = 10
train_data[:block_size+1]
#this simulates a basic ngram proposition, where 50 implies 18. 50 and 18 implies 18. 50, 18, and 18 implies 63. etc.
#this trains the transformer to be used to context of n=1 all the way up to n=block_size.

tensor([16,  9,  9, 19, 52, 10, 31, 47, 32, 61, 53])

In [12]:
#blocks/chunks are groups of tokens
#batches are groups of blocks/chunks

In [13]:
torch.manual_seed(0)
block_size = 10
batch_size = 4
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(0,len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [14]:
xb,yb = get_batch('train')
print(xb.shape,'\n',xb)
print(yb.shape,'\n',yb)
for b in range(batch_size):
    for c in range(block_size):
        con = xb[b, :c+1]
        tar = yb[b, c]
        print("When input is", con, "target is", tar)

torch.Size([4, 10]) 
 tensor([[10,  8, 22, 10, 61, 24, 61, 48, 19, 32],
        [ 9, 10, 22, 10, 19, 12, 10, 19, 27, 53],
        [11, 48, 12, 50,  9, 22, 16, 27, 39, 48],
        [32,  1, 12, 48, 32, 48, 16, 63, 10, 19]])
torch.Size([4, 10]) 
 tensor([[ 8, 22, 10, 61, 24, 61, 48, 19, 32, 22],
        [10, 22, 10, 19, 12, 10, 19, 27, 53, 49],
        [48, 12, 50,  9, 22, 16, 27, 39, 48, 36],
        [ 1, 12, 48, 32, 48, 16, 63, 10, 19, 12]])
When input is tensor([10]) target is tensor(8)
When input is tensor([10,  8]) target is tensor(22)
When input is tensor([10,  8, 22]) target is tensor(10)
When input is tensor([10,  8, 22, 10]) target is tensor(61)
When input is tensor([10,  8, 22, 10, 61]) target is tensor(24)
When input is tensor([10,  8, 22, 10, 61, 24]) target is tensor(61)
When input is tensor([10,  8, 22, 10, 61, 24, 61]) target is tensor(48)
When input is tensor([10,  8, 22, 10, 61, 24, 61, 48]) target is tensor(19)
When input is tensor([10,  8, 22, 10, 61, 24, 61, 48, 19]) 

In [15]:
import torch.nn as nn
import torch.nn.functional as F
class Bigram(nn.Module):
    def __init__(self,  vocab):
        super().__init__()
        self.token_embedding_table= nn.Embedding(vocab, vocab)
    def forward(self, idx, targets=None):
        logits= self.token_embedding_table(idx)#creates a [batch, chunk, vocab] tensor
            #logits represent the next scores for each embedding
       
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)#reshaping for the loss fn
            targets = targets.view(-1)#reshaping for the loss fn
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max):
        #idx is the current token
        for _ in range(max):
            #get predictions
            logits, loss = self(idx)
            logits = logits[:,-1,:]#get the timestep to form (B,C) tensor
            probs= F.softmax(logits, dim=-1)
            next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next), dim=1)
        return idx
m = Bigram(vocab)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([40, 65])
tensor(4.8956, grad_fn=<NllLossBackward0>)


In [16]:
#testing
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))

;!O
B:W'iwLVNf$sDNh:t
cG!qJjurUEmqqZH,ez:;c!o3xOz,QKUZBdEjIFz??,G?,v!Ep&kHOFPZlfwnmhZE-,in.fhmhpYDjpO


In [17]:
#training
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [18]:
batch_size=32
for i in range(10000):
    x,y = get_batch('train')
    logits, loss = m(x, y)
    optimizer.zero_grad(set_to_none=True)#set grads to 0
    loss.backward()#do backward prop
    optimizer.step()#update params based on grad values
print(loss.item())

3.234619617462158


In [19]:
print(decode(m.generate(idx, 100)[0].tolist()))

;E aowPuInh
V, camnrrohesaamaOasnoAn
 an N'd, ienent rvamwt hlhnlIweo ieCGe;,?h ouf,oxa yoteU-Oaf!yrn


In [20]:
torch.manual_seed(0)
B, T,C = 8,4,2
x = torch.randn(B,T,C) #b is batches, t is chunks, c is vocab size
x.shape

torch.Size([8, 4, 2])

In [21]:
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] 
        xbow[b, t] = torch.mean(xprev, 0)

In [22]:
xbow[0]

tensor([[-1.1258, -1.1524],
        [-0.6882, -0.7931],
        [-0.1759, -0.2981],
        [-0.2109, -0.7524]])

In [23]:
#@can be used for matrix multiplication
#more efficient way of memory using matrix averaging.
tril = torch.tril(torch.ones(T, T))
weights = torch.tril(torch.zeros(T, T)) #lower triange matrix
weights = weights/weights.sum(1, keepdim=True)#make each row sum to 1, as probabilities should be from 0 to 1
xbow2 = weights@x

In [24]:
weights = torch.tril(torch.ones(T, T)) #lower triange matrix
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)
weights #again, same thing, but uses a softmax function, which exponentiates the values and then normalizes them
xbow3 = weights@x
xbow3
#basically, the weights are used to represent each time step as a weighted average of all the previous time steps

tensor([[[-1.1258, -1.1524],
         [-0.6882, -0.7931],
         [-0.1759, -0.2981],
         [-0.2109, -0.7524]],

        [[ 0.3223, -1.2633],
         [ 0.3361, -0.4776],
         [ 0.2640,  0.0942],
         [ 0.4772,  0.0088]],

        [[-1.3527, -1.6959],
         [-0.3930, -0.4512],
         [-0.0624, -0.8192],
         [-0.1321, -0.1511]],

        [[ 0.7502, -0.5855],
         [ 0.2884, -0.2010],
         [ 0.6554,  0.3948],
         [ 0.7281,  0.0852]],

        [[-0.6136,  0.0316],
         [-0.5531,  0.1400],
         [-0.2222,  0.1308],
         [-0.0064,  0.2084]],

        [[-0.1023,  0.7924],
         [-0.1960,  0.4225],
         [ 0.0436,  1.0491],
         [-0.3345,  0.3901]],

        [[-0.6731,  0.8728],
         [ 0.1911,  0.5253],
         [ 0.0506,  0.2196],
         [ 0.1738,  0.0659]],

        [[-0.4462,  0.7440],
         [ 0.5374,  2.0773],
         [-0.1521,  0.9735],
         [ 0.3408,  0.5922]]])

In [31]:
#self-attention: works by giving memory in a data dependent way. Every token in every position emits 2 vectors:
#query and key. The query vector,"what am i looking for", is compared to every key vector to get a score. 
#the key vector, 'what do i contain', is used to perform a dot product with the query vector. the dot product becomes the weight.

#using a single head:
head_size = 16
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False) #simple dot product
value = nn.Linear(C, head_size, bias=False)
k = key(x) # [B, T, 16]
q = query(x)# [B, T, 16]
v = value(x)# [B, T, 16]
wei = q @ k.transpose(-2, -1) # [B, T, 16]@[B, 16, T] -> [B, T, T] this gives the affinity score
tril = torch.tril(torch.ones(T, T)) # [T, T] lower triangular matrix
wei = wei.masked_fill(tril == 0, float('-inf')) # [B, T, T] masking the upper triangular matrix to disallow future tokens
wei = F.softmax(wei, dim=-1) # [B, T, T] softmax along the last dimension
output = wei @ v # [B, T, T]@[B, T, C] -> [B, T, C] this gives the weighted sum of the values
output.shape

torch.Size([4, 8, 16])

In [32]:
wei[0]

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.2854e-02, 9.5715e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [4.7353e-02, 5.8926e-02, 8.9372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.0357e-01, 1.4476e-01, 6.1413e-01, 3.7541e-02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [2.0271e-01, 1.7030e-01, 1.2484e-01, 8.8532e-02, 4.1362e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.4038e-01, 2.0584e-02, 5.9102e-01, 3.9057e-02, 1.8155e-01, 2.7407e-02,
         0.0000e+00, 0.0000e+00],
        [7.8017e-02, 5.4403e-01, 3.2077e-02, 1.9174e-01, 8.4955e-03, 6.5421e-02,
         8.0217e-02, 0.0000e+00],
        [5.6004e-02, 1.8383e-02, 4.3535e-04, 4.4384e-02, 3.3305e-02, 7.5200e-03,
         2.5908e-02, 8.1406e-01]], grad_fn=<SelectBackward0>)

In [3]:
def partition_minimized_difference(arr):
    arr.sort()
    n = len(arr)
    
    # Initialize three subarrays
    subarrays = [[] for _ in range(3)]
    
    # Distribute elements while minimizing differences
    for i in range(n - 1, -1, -1):
        min_subarray = min(subarrays, key=lambda subarray: sum(subarray))
        min_subarray.append(arr[i])
    
    return subarrays

# Example usage
nums = [5, 7, 10, 12, 100, 550, 1000]
subarrays = partition_minimized_difference(nums)
for i, subarray in enumerate(subarrays):
    print(f"Subarray {i + 1}:", subarray)

Subarray 1: [1000]
Subarray 2: [550]
Subarray 3: [100, 12, 10, 7, 5]
